In [1]:
import requests
from typing import Dict, Any, Optional
import geopandas as gpd
from geopandas import GeoDataFrame

BASE_URL = "http://10.32.1.107:5300/api/v1"

def get_functional_zones(
    territory_id: int,
    year: Optional[int] = None,
    source: Optional[str] = None,
    functional_zone_type_id: Optional[int] = None,
    include_child_territories: bool = False,
    cities_only: bool = False
) -> GeoDataFrame:
    """
    Получение функциональных зон для указанной территории в виде GeoDataFrame
    """
    url = f"{BASE_URL}/territory/{territory_id}/functional_zones"
    
    # Параметры запроса
    params = {
        "include_child_territories": include_child_territories,
        "cities_only": cities_only,
        "source":'OSM'
    }
    
    if year is not None:
        params["year"] = year
    if functional_zone_type_id is not None:
        params["functional_zone_type_id"] = functional_zone_type_id

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        # Предполагаем, что API возвращает список словарей с геометрией
        data = response.json()
        # Преобразуем в GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(data) if data else gpd.GeoDataFrame()
        return gdf
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе функциональных зон: {e}")
        return gpd.GeoDataFrame()
    except ValueError as e:
        print(f"Ошибка при преобразовании в GeoDataFrame: {e}")
        return gpd.GeoDataFrame()

def get_all_territories(
    parent_id: Optional[int] = None,
    get_all_levels: bool = False,
    territory_type_id: Optional[int] = None,
    name: Optional[str] = None,
    cities_only: bool = False,
    created_at: Optional[str] = None,
    centers_only: bool = False
) -> GeoDataFrame:
    """
    Получение списка всех территорий в виде GeoDataFrame
    """
    url = f"{BASE_URL}/all_territories"
    
    # Параметры запроса
    params = {
        "get_all_levels": get_all_levels,
        "cities_only": cities_only,
        "centers_only": centers_only
    }
    
    if parent_id is not None:
        params["parent_id"] = parent_id
    if territory_type_id is not None:
        params["territory_type_id"] = territory_type_id
    if name is not None:
        params["name"] = name
    if created_at is not None:
        params["created_at"] = created_at

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        # Предполагаем, что API возвращает GeoJSON
        data = response.json()
        # Преобразуем GeoJSON в GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(data["features"]) if data.get("features") else gpd.GeoDataFrame()
        return gdf
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе территорий: {e}")
        return gpd.GeoDataFrame()
    except ValueError as e:
        print(f"Ошибка при преобразовании в GeoDataFrame: {e}")
        return gpd.GeoDataFrame()

In [2]:
# Пример 1: Получение функциональных зон для территории с ID 1
zones_gdf = get_functional_zones(
    territory_id=1,
    year=2025,
    source="OSM",
    include_child_territories=True
)
print("Функциональные зоны (GeoDataFrame):")
zones_gdf.head()


Функциональные зоны (GeoDataFrame):


,geometry,landuse_zon,id
0,"POLYGON ((30.10762 58.6112, 30.10865 58.61039,...",Residential,NaN
1,"POLYGON ((30.38304 60.05619, 30.38323 60.05639...",Business,NaN
2,"POLYGON ((30.38514 60.05692, 30.38478 60.05674...",Recreation,NaN
3,"POLYGON ((30.38084 60.05327, 30.38077 60.05328...",Recreation,NaN
4,"POLYGON ((30.38565 60.05967, 30.38667 60.05938...",Recreation,NaN


In [3]:
zones_gdf = zones_gdf.set_crs(4326)

In [4]:
zones_gdf.explore()

In [5]:
# territories_gdf = get_all_territories(
#     parent_id=1,
#     get_all_levels=True,
#     cities_only=False
# )
# territories_gdf.to_parquet('territories_gdf.parquet')

In [6]:
territories_gdf = gpd.read_parquet('territories_gdf.parquet')

In [7]:
gpsp_gdf = territories_gdf[territories_gdf['level'] == 4]

In [8]:
import requests
from typing import Optional
import geopandas as gpd
from geopandas import GeoDataFrame

BASE_URL = "http://10.32.1.107:5300/api/v1"

def get_physical_objects_geojson(
    territory_id: int,
) -> GeoDataFrame:

    url = f"http://10.32.1.107:5300/api/v1/territory/{territory_id}/physical_objects_geojson?include_child_territories=false&cities_only=false&centers_only=false&physical_object_function_id=1"

    try:
        response = requests.get(url)
        response.raise_for_status()
        
        # Предполагаем, что API возвращает GeoJSON с полем "features"
        data = response.json()
        features = data.get("features", [])
        
        if not features:
            return gpd.GeoDataFrame()
            
        gdf = gpd.GeoDataFrame.from_features(features)
        gdf['territory_id'] = territory_id
        return gdf
    except:
        print(f"Ошибка при запросе")
        return gpd.GeoDataFrame()


In [ ]:
# from tqdm import tqdm

# gpsp_ids = gpsp_gdf['territory_id'].unique()
# physical_objects_gdf_list = []
# for id in tqdm(gpsp_ids):
#     physical_objects_gdf_list.append(get_physical_objects_geojson(territory_id=id))
# buildings_gdf = gpd.pd.concat(physical_objects_gdf_list, axis=0, ignore_index=True)
# buildings_gdf.to_parquet('buildings_gdf.parquet')

  0%|          | 0/188 [00:00<?, ?it/s]

100%|██████████| 188/188 [04:05<00:00,  1.31s/it]


In [57]:
buildings_gdf = gpd.read_parquet('buildings_gdf.parquet')

In [44]:
buildings_gdf = buildings_gdf[buildings_gdf['building'].notna()]

In [58]:
buildings_gdf = buildings_gdf.join(gpd.pd.json_normalize(buildings_gdf['building']))
buildings_gdf = buildings_gdf.drop(columns=['building']) 

living_building_type_id = 4
key_to_check = 'physical_object_type_id'  
buildings_gdf['is_living'] = buildings_gdf.apply(lambda row: row['physical_object_type'].get('physical_object_type_id') == living_building_type_id, axis=1)

In [65]:
columns = ['geometry','territory_id','is_living','building_area_official',
           'properties.storeys_count','floors',
           'properties.population_balanced',]

buildings_gdf = buildings_gdf[columns]

In [66]:
buildings_gdf['is_living'].sum()

np.int64(17583)

In [71]:
# Разделяем DataFrame на два по значению is_living
true_df = buildings_gdf[buildings_gdf['is_living'] == True]
false_df = buildings_gdf[buildings_gdf['is_living'] == False]

# Считаем количество не-NaN значений для каждой группы
columns_to_count = [
    'building_area_official',
    'properties.storeys_count',
    'floors',
    'properties.population_balanced'
]

true_counts = true_df[columns_to_count].count()
false_counts = false_df[columns_to_count].count()

# Выводим результат
print("Количество ненулевых значений для is_living == True:")
print(true_counts)
print("\nКоличество ненулевых значений для is_living == False:")
print(false_counts)

# Если нужно общее количество строк в каждой группе
print(f"\nВсего строк с is_living == True: {len(true_df)}")
print(f"Всего строк с is_living == False: {len(false_df)}")

Количество ненулевых значений для is_living == True:
building_area_official            10613
properties.storeys_count            790
floors                              918
properties.population_balanced    10613
dtype: int64

Количество ненулевых значений для is_living == False:
building_area_official            0
properties.storeys_count          0
floors                            0
properties.population_balanced    0
dtype: int64

Всего строк с is_living == True: 17583
Всего строк с is_living == False: 134338


In [26]:
import geopandas as gpd
import pandas as pd

# Отделяем агрегацию числовых колонок
numeric_agg_dict = {
    'building_area_modeled': ['sum', 'mean'],
    'properties.storeys_count': ['sum', 'mean'],
    'properties.population_balanced': ['sum', 'mean'],
    'properties.living_area_modeled': ['sum', 'mean'],
    'floors': ['sum', 'mean']
}

# Агрегируем числовые колонки
numeric_agg = buildings_gdf.groupby('territory_id').agg(numeric_agg_dict)

# Исправляем названия колонок для числовых данных
numeric_agg.columns = ['_'.join(col).strip() for col in numeric_agg.columns.values]

# Получаем первые значения для нечисловых колонок
first_values = buildings_gdf.groupby('territory_id').agg({
    'geometry': 'first',
    'is_living': 'first'
}).reset_index()

# Объединяем результаты
aggregated_gdf = first_values.merge(numeric_agg.reset_index(), on='territory_id')

# Преобразуем обратно в GeoDataFrame
aggregated_gdf = gpd.GeoDataFrame(aggregated_gdf, geometry='geometry')

# Если нужно сохранить CRS из исходного gdf
if buildings_gdf.crs:
    aggregated_gdf.crs = buildings_gdf.crs

In [27]:
aggregated_gdf

,territory_id,geometry,is_living,building_area_modeled_sum,building_area_modeled_mean,properties.storeys_count_sum,properties.storeys_count_mean,properties.population_balanced_sum,properties.population_balanced_mean,properties.living_area_modeled_sum,properties.living_area_modeled_mean,floors_sum,floors_mean
0,3,POINT (34.17661 59.58829),False,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,4,"POLYGON ((33.73065 59.61149, 33.73075 59.61148...",False,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,5,POINT (34.19053 59.49318),False,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,6,"POLYGON ((33.7432 59.46092, 33.74321 59.46086,...",True,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,7,"POLYGON ((33.86278 59.49476, 33.86281 59.49467...",False,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,203,POINT (31.02367 59.62673),False,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
163,204,POINT (30.65466 59.66402),False,0.0,0.0,6.0,0.050420,0.0,0.0,0.0,0.0,6.0,0.050420
164,205,POINT (31.45179 59.20752),False,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
165,206,POINT (30.87941 59.10976),False,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
